In [14]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [15]:
# 저장한 데이터 불러오기
df = pd.read_excel('(1)samsung_all_buds.xlsx')
df.head(20)

,ratings,reviews,label
0,5,"가성비 갑, 성능 좋아요!! 추천드립니다! **내돈내산 후기입니다**구매 및 개봉일...",1
1,5,전직 컴수리기사의 버즈fe 테스트 결과보고서 주 문 : 24. 3. 27도 착 :...,1
2,5,가성비 그자체의 무선 이어폰입니다 내돈내산 솔직후기!!!---------------...,1
3,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품 작년 겨울에 쿠팡에서 산 저렴...,1
4,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고! (づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버...",1
5,5,가성비 굿 신상 최고 안녕하세요제가 버즈시리즈는 전부 가지고 있는 사람입니다!버즈 ...,1
6,5,가성비 커널형 이어폰 ✅️ 갤럭시 버즈 FE 사용하는 이유:✅️1. 편리한 무선 사...,1
7,5,가격대비 성능 좋은 보급형 무선이어폰 #내돈내산후기#버즈 2를 사용하다가 한쪽 유닛...,1
8,5,등록된 헤드라인이 없습니다 늦깎이 대학생인 남편 위해 구매했어요-----------...,1
9,5,[가성비의 끝판왕] 삼성전자 갤럭시 버즈 FE 후기 [가성비의 끝판왕] 삼성전자 갤...,1


In [17]:
positiveDf = df[df['label'] == 1]
positiveDf

,ratings,reviews,label
0,5,"가성비 갑, 성능 좋아요!! 추천드립니다! **내돈내산 후기입니다**구매 및 개봉일...",1
1,5,전직 컴수리기사의 버즈fe 테스트 결과보고서 주 문 : 24. 3. 27도 착 :...,1
2,5,가성비 그자체의 무선 이어폰입니다 내돈내산 솔직후기!!!---------------...,1
3,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품 작년 겨울에 쿠팡에서 산 저렴...,1
4,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고! (づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버...",1
...,...,...,...
6795,4,등록된 헤드라인이 없습니다 사운드 품질은 생각보다 좀 떨어지는듯 싶고 갤럭시 웨어러...,1
6796,4,"만족 다행히 페어링 불량, 오류난거 없었구요 사용 잘 하구 있습니다 박스안에 덩그러...",1
6797,5,자주 구매할 것 같아요. 불량사항 없습니다. 아버지가 너무 좋아하시더라구요.쿠팡으로...,1
6798,4,선으로부터의 자유...앞으로 기대됩니다~^^ 색상은 예상했던대로 산뜻하니 세련되보이...,1


In [18]:
# 리뷰 데이터만 가져오기
review = positiveDf['reviews']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력


In [19]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [20]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [32]:
# '등록 헤드라인 등록 리뷰 용이'는 리뷰가 없으면 자동생성되는 내용임 
# '버즈'에 대한 크롤링 내용이기에 압도적으로 많을 수 뿐이 없기에 이러한 필요없는 명사 삭제처리
removeWords = ['등록', '헤드라인', '리뷰', '용이', '제품', '버즈']

ids_to_remove = [dict.token2id[word] for word in removeWords if word in dict.token2id]
dict.filter_tokens(bad_ids=ids_to_remove)

In [33]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

# '등록 헤드라인 등록 리뷰 용이'는 리뷰가 없으면 자동생성되는 내용임 
# '버즈'에 대한 크롤링 내용이기에 압도적으로 많을 수 뿐이 없기에 이러한 필요없는 명사 삭제처리
removeWords = ['등록', '헤드라인', '리뷰', '용이', '제품', '에어팟']

ids_to_remove = [dict.token2id[word] for word in removeWords if word in dict.token2id]
dict.filter_tokens(bad_ids=ids_to_remove)

# 딕셔너리화 성공~
print(dict.token2id)

{'가격': 0, '가성': 1, '개봉': 2, '개봉일': 3, '개인': 4, '갤럭시': 5, '경우': 6, '고무': 7, '고민': 8, '과소': 9, '구매': 10, '귀가': 11, '그레이': 12, '기능': 13, '기존': 14, '기차': 15, '내부': 16, '노이즈': 17, '다만': 18, '다음': 19, '더욱': 20, '도움': 21, '도착': 22, '뒤쪽': 23, '등등': 24, '때문': 25, '말소리': 26, '모두': 27, '방송': 28, '배터리': 29, '버드': 30, '버튼': 31, '변경': 32, '별로': 33, '보급': 34, '보조': 35, '본체': 36, '부분': 37, '분실': 38, '브랜드': 39, '블랙': 40, '블루투스': 41, '비일': 42, '뽁뽁이': 43, '사고': 44, '사람': 45, '사용': 46, '사용자': 47, '사유': 48, '사진': 49, '상자': 50, '생각': 51, '선택': 52, '설명': 53, '성능': 54, '성품': 55, '슬링': 56, '시간': 57, '실제': 58, '쓰기': 59, '양쪽': 60, '어댑터': 61, '여행': 62, '연한': 63, '오히려': 64, '완전': 65, '우선': 66, '위아래': 67, '음질': 68, '이어폰': 69, '인식': 70, '일단': 71, '일이': 72, '자주': 73, '전용': 74, '정도': 75, '정말': 76, '제거': 77, '제외': 78, '준수': 79, '직접': 80, '차안': 81, '차이': 82, '차이나': 83, '착용': 84, '첨부': 85, '추천': 86, '출발': 87, '충전': 88, '측면': 89, '컴퓨터': 90, '케이스': 91, '타입': 92, '택배': 93, '패킹': 94, '편이': 95, '한쪽': 96, '합리': 97, '해당': 98, '

In [34]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [35]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 3), (1, 3), (2, 2), (3, 1), (4, 1), (5, 4), (6, 2), (7, 2), (8, 2), (9, 1), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 4), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 4), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 5), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 2), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 2), (70, 2), (71, 1), (72, 1), (73, 1), (74, 2), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 2), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 3), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 3), (102, 1)], [(0, 4), (1, 3), (5, 4), (11, 3), (21, 2), (22, 1), (24, 2), (27, 1), (29, 2)

In [36]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [37]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.023*"구매" + 0.017*"음질" + 0.016*"사용" + 0.013*"이어폰" + 0.013*"착용" + 0.013*"프로" + 0.012*"통화" + 0.011*"소리" + 0.010*"케이스" + 0.010*"가격" + 0.009*"플러스" + 0.009*"생각" + 0.009*"삼성" + 0.009*"기능" + 0.008*"갤럭시"'), (1, '0.030*"사용" + 0.027*"이어폰" + 0.023*"음질" + 0.019*"갤럭시" + 0.018*"착용" + 0.017*"슬링" + 0.016*"노이즈" + 0.014*"구매" + 0.014*"프로" + 0.014*"소리" + 0.013*"에어팟" + 0.011*"라이브" + 0.010*"기능" + 0.008*"선물" + 0.008*"플러스"'), (2, '0.025*"음질" + 0.021*"이어폰" + 0.020*"사용" + 0.017*"프로" + 0.016*"착용" + 0.016*"가격" + 0.013*"소리" + 0.010*"구매" + 0.009*"배송" + 0.009*"에어팟" + 0.009*"기능" + 0.009*"선물" + 0.009*"귀가" + 0.008*"노이즈" + 0.008*"갤럭시"')]


In [38]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)

### 토픽분석을 통해 얻을 수 있는 정보
1. 삼성에서는 애플에 에어팟에 비교하는 댓글들이 많음
2. 애플의 에어팟에 비하여 가격이 저렴하기에 구매하는 경우가 많음
3. 주로 가격이 크게 비싸지 않기에 다른 사람에게 선물을 한다는 리뷰가 많음